In [1]:
import pandas as pd
import pathlib as pl
import subprocess

In [2]:
dry_run = True

In [3]:
def in_notebook():
    try:
        from IPython import get_ipython
        if 'IPKernelApp' not in get_ipython().config:  # pragma: no cover
            return False
    except ImportError:
        return False
    except AttributeError:
        # In a terminal/shell, get_ipython() will return None
        return False
    return True

In [4]:
here = pl.Path.cwd() if in_notebook() else pl.Path(__file__).parent

with (here / 'FIG_VERSION').open('r') as file:
    version = file.read().strip()

with (here / 'DIR_NAME_FMT').open('r') as file:
    dir_name_fmt = file.read().strip()

with (here / 'FIG_NAME_FMT').open('r') as file:
    fig_name_fmt = file.read().strip()
    
alias_table = pd.read_csv(here / 'figure_aliases_updated.csv')
alias_to_number = alias_table.set_index('alias')[version].to_dict()
number_to_alias = alias_table.set_index(version)['alias'].to_dict()
alias_to_new_alias = alias_table.set_index('alias')['new_alias'].to_dict()

parent_directories = list(here.glob('fig_*'))

In [5]:
def rename_directory(d, dry_run=dry_run):
    dir_parts = d.name.split('__')
    alias = dir_parts[1]
    number = alias_to_number[alias]
    new_alias = alias_to_new_alias[alias]
    new_name = dir_name_fmt.format(number=number, alias=new_alias)
    git_mv_cmd = ['git', 'mv', str(d), str(d.with_name(new_name))]
    if new_name == d.name:
        print(f"Directory {d.name} already has the correct name.")
        return new_name
    print(' '.join(git_mv_cmd))
    if not dry_run:
        subprocess.run(git_mv_cmd)
        
    return new_name
        
def rename_file(f, dry_run=dry_run):
    fig_parts = f.stem.split('__')
    assert len(fig_parts) <= 3, f"Unexpected file naming format: {f.name}"
    alias = fig_parts[1]
    number = alias_to_number[alias]
    if len(fig_parts) == 3:
        extra = '__' + fig_parts[2]
    else:
        extra = ''
    new_alias = alias_to_new_alias[alias]
    new_name = fig_name_fmt.format(number=number, alias=new_alias, extra=extra)
    git_mv_cmd = ['git', 'mv', str(f), str(f.with_stem(new_name))]
    if new_name == f.stem:
        print(f"File {f.name} already has the correct name.")
        return new_name
    print(' '.join(git_mv_cmd))
    if not dry_run:
        subprocess.run(git_mv_cmd)
    return new_name
    
for dry_run in [True, False]:
    # Do a dry run first to ensure there are no errors
    print(f"{'dry run' if dry_run else 'actual run'} mode...\n")
    for d in parent_directories:
        assert d.is_dir(), f"{d} is not a directory"
        new_dir_name = rename_directory(d, dry_run=dry_run)
        
        if not dry_run:
            d = d.with_name(new_dir_name)

        for ext in ['.png', '.svg', '.ipynb']:
            for f in d.rglob(f'*{ext}'):
                if not f.is_file():
                    continue
                rename_file(f, dry_run=dry_run)
        print()

dry run mode...

git mv /home/john/projects/can-manuscript/fig_1__glassy_desert /home/john/projects/can-manuscript/fig_1__sculpted_antelope
git mv /home/john/projects/can-manuscript/fig_1__glassy_desert/graphics/fig_1Av__square_melon.png /home/john/projects/can-manuscript/fig_1__glassy_desert/graphics/fig_1Av__silent_fox.png
git mv /home/john/projects/can-manuscript/fig_1__glassy_desert/graphics/fig_1Aiii__transparent_television.png /home/john/projects/can-manuscript/fig_1__glassy_desert/graphics/fig_1Aiii__flexible_dragonfly.png
git mv /home/john/projects/can-manuscript/fig_1__glassy_desert/graphics/fig_1Div__wooden_internet.png /home/john/projects/can-manuscript/fig_1__glassy_desert/graphics/fig_1Div__wooden_quail.png
git mv /home/john/projects/can-manuscript/fig_1__glassy_desert/graphics/fig_1Dii__bronze_thought.png /home/john/projects/can-manuscript/fig_1__glassy_desert/graphics/fig_1Dii__bronze_ocelot.png
git mv /home/john/projects/can-manuscript/fig_1__glassy_desert/graphics/fig_